In [10]:
import geopandas as gpd
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_PG       #from local site-packages
from geoalchemy2 import Geometry
import shapely
from bs4 import BeautifulSoup
import numpy as np

engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)

sql_alc_cnxn = engine.connect()

taxis = 'data/TaxiStand_Jan2022/TaxiStop.shp'
roads = 'data/national-map-line/national-map-line-geojson.geojson'
busstops = 'data/BusStopLocation_Jan2022/BusStop.shp'
trains = 'data/TrainStation_Jan2022/MRTLRTStnPtt.shp'
exits = 'data/TrainStationExit_Aug2021/Train_Station_Exit_Layer.shp'

## Load and Process Taxi Stands

In [4]:
# read file
taxi_df = gpd.read_file(taxis)
taxi_df

,TYPE_CD,TYPE_CD_DE,geometry
0,None,TAXI STOP,POINT (29901.825 30391.618)
1,None,TAXI STOP,POINT (30179.979 30762.080)
2,None,TAXI STOP,POINT (29654.555 30748.886)
3,None,TAXI STAND,POINT (28932.411 29355.413)
4,None,TAXI STAND,POINT (28814.750 29203.531)
...,...,...,...
349,None,TAXI STAND,POINT (33746.473 40585.611)
350,None,TAXI STAND,POINT (31288.758 32246.030)
351,None,TAXI STOP,POINT (31260.830 32119.016)
352,None,TAXI STAND,POINT (30408.027 33888.227)


### Check CRS and convert 

In [5]:
# check format
taxi_df.crs

<Projected CRS: PROJCS["SVY21",GEOGCS["GCS_WGS_1984",DATUM["WGS_19 ...>
Name: SVY21
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS_84
- Prime Meridian: Greenwich

In [6]:
# SVY21 is EPSG 3414
# we convert this to 4326
taxi_df = taxi_df.to_crs(4326)

# we also swap around lat longs
taxi_df['geometry']=taxi_df.geometry.map(lambda point: shapely.ops.transform(lambda x, y: (y, x), point))
taxi_df

### Upload to DB

In [8]:
taxi_df.to_postgis('taxi_stands',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})

# Load and Process Bus Stops

In [11]:
# read file
bus_df = gpd.read_file(busstops)
bus_df

,BUS_STOP_N,BUS_ROOF_N,LOC_DESC,geometry
0,22069,B06,OPP CEVA LOGISTICS,POINT (13576.312 32883.655)
1,32071,B23,AFT TRACK 13,POINT (13228.592 44206.377)
2,44331,B01,BLK 239,POINT (21045.101 40242.079)
3,96081,B05,GRACE INDEPENDENT CH,POINT (41603.764 35413.109)
4,11561,B05,BLK 166,POINT (24568.738 30391.846)
...,...,...,...,...
5154,52311,B01,BLK 23,POINT (30666.090 35170.408)
5155,17061,B09,BLK 701,POINT (20033.571 32233.988)
5156,03222,B01,HUB SYNERGY PT,POINT (29292.040 28502.066)
5157,26419,NIL,AFT TUAS VIEW CIRCUIT,POINT (5716.847 30148.318)


In [12]:
# check format
bus_df.crs

<Projected CRS: PROJCS["SVY21",GEOGCS["GCS_WGS_1984",DATUM["WGS_19 ...>
Name: SVY21
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS_84
- Prime Meridian: Greenwich

In [13]:
# SVY21 is EPSG 3414
# we convert this to 4326
bus_df = bus_df.to_crs(4326)

# we also swap around lat longs
bus_df['geometry']=bus_df.geometry.map(lambda point: shapely.ops.transform(lambda x, y: (y, x), point))
bus_df

,BUS_STOP_N,BUS_ROOF_N,LOC_DESC,geometry
0,22069,B06,OPP CEVA LOGISTICS,POINT (1.314 103.704)
1,32071,B23,AFT TRACK 13,POINT (1.416 103.701)
2,44331,B01,BLK 239,POINT (1.380 103.771)
3,96081,B05,GRACE INDEPENDENT CH,POINT (1.337 103.956)
4,11561,B05,BLK 166,POINT (1.291 103.802)
...,...,...,...,...
5154,52311,B01,BLK 23,POINT (1.334 103.857)
5155,17061,B09,BLK 701,POINT (1.308 103.762)
5156,03222,B01,HUB SYNERGY PT,POINT (1.274 103.845)
5157,26419,NIL,AFT TUAS VIEW CIRCUIT,POINT (1.289 103.633)


### Upload to Database

In [14]:
bus_df.to_postgis('bus_stops',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})

# Load and Process MRT Stations

In [15]:
# read file
trains_df = gpd.read_file(trains)
trains_df

,STN_NAME,STN_NO,geometry
0,ESPLANADE MRT STATION,CC3,POINT (30481.034 30627.639)
1,PAYA LEBAR MRT STATION,EW8/CC9,POINT (34560.965 33293.610)
2,DHOBY GHAUT MRT STATION,NS24/NE6/CC1,POINT (29392.761 31267.230)
3,DAKOTA MRT STATION,CC8,POINT (34159.908 32299.063)
4,LAVENDER MRT STATION,EW11,POINT (31285.215 32188.061)
...,...,...,...
186,TANAH MERAH MRT STATION,EW4,POINT (40601.284 34386.818)
187,TAN KAH KEE MRT STATION,DT8,POINT (25107.830 34233.567)
188,UPPER THOMSON MRT STATION,TE8,POINT (27956.061 37388.745)
189,CALDECOTT MRT STATION,CC17/TE9,POINT (28758.983 35499.264)


In [32]:
# add aditional columns
trains_df['TYPE'] = trains_df['STN_NAME'].str.split().str[-2]
trains_df['NO_OF_LINES'] = trains_df['STN_NO'].str.count("/")+1
trains_df['STAION_SCORE'] = np.where(trains_df['TYPE']=='MRT',1,0.5) + trains_df['NO_OF_LINES']
trains_df

,STN_NAME,STN_NO,geometry,TYPE,NO_OF_LINES,STAION_SCORE
0,ESPLANADE MRT STATION,CC3,POINT (30481.034 30627.639),MRT,1,2.0
1,PAYA LEBAR MRT STATION,EW8/CC9,POINT (34560.965 33293.610),MRT,2,3.0
2,DHOBY GHAUT MRT STATION,NS24/NE6/CC1,POINT (29392.761 31267.230),MRT,3,4.0
3,DAKOTA MRT STATION,CC8,POINT (34159.908 32299.063),MRT,1,2.0
4,LAVENDER MRT STATION,EW11,POINT (31285.215 32188.061),MRT,1,2.0
...,...,...,...,...,...,...
186,TANAH MERAH MRT STATION,EW4,POINT (40601.284 34386.818),MRT,1,2.0
187,TAN KAH KEE MRT STATION,DT8,POINT (25107.830 34233.567),MRT,1,2.0
188,UPPER THOMSON MRT STATION,TE8,POINT (27956.061 37388.745),MRT,1,2.0
189,CALDECOTT MRT STATION,CC17/TE9,POINT (28758.983 35499.264),MRT,2,3.0


In [33]:
trains_df.crs

<Projected CRS: PROJCS["SVY21",GEOGCS["GCS_WGS_1984",DATUM["WGS_19 ...>
Name: SVY21
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS_84
- Prime Meridian: Greenwich

In [34]:
# SVY21 is EPSG 3414
# we convert this to 4326
trains_df = trains_df.to_crs(4326)

# we also swap around lat longs
trains_df['geometry']=trains_df.geometry.map(lambda point: shapely.ops.transform(lambda x, y: (y, x), point))
trains_df

,STN_NAME,STN_NO,geometry,TYPE,NO_OF_LINES,STAION_SCORE
0,ESPLANADE MRT STATION,CC3,POINT (1.293 103.856),MRT,1,2.0
1,PAYA LEBAR MRT STATION,EW8/CC9,POINT (1.317 103.892),MRT,2,3.0
2,DHOBY GHAUT MRT STATION,NS24/NE6/CC1,POINT (1.299 103.846),MRT,3,4.0
3,DAKOTA MRT STATION,CC8,POINT (1.308 103.889),MRT,1,2.0
4,LAVENDER MRT STATION,EW11,POINT (1.307 103.863),MRT,1,2.0
...,...,...,...,...,...,...
186,TANAH MERAH MRT STATION,EW4,POINT (1.327 103.947),MRT,1,2.0
187,TAN KAH KEE MRT STATION,DT8,POINT (1.326 103.807),MRT,1,2.0
188,UPPER THOMSON MRT STATION,TE8,POINT (1.354 103.833),MRT,1,2.0
189,CALDECOTT MRT STATION,CC17/TE9,POINT (1.337 103.840),MRT,2,3.0


### Upload to Database

In [35]:
trains_df.to_postgis('transit_stations',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})

## Load and Process Trains Exits

In [36]:
# read file
exits_df = gpd.read_file(exits)
exits_df

,EXIT_CODE,EXIT_CODE_,geometry
0,None,Exit A,POINT (34285.068 34322.985)
1,None,Exit B,POINT (34382.153 34231.904)
2,None,Exit C,POINT (34337.292 34190.603)
3,None,Exit B,POINT (33872.145 41256.053)
4,None,Exit A,POINT (33858.541 41234.065)
...,...,...,...
474,None,Exit E,POINT (42334.632 35228.295)
475,None,Exit D,POINT (34136.110 34153.437)
476,None,Exit G,POINT (42437.644 35245.551)
477,None,Exit A,POINT (42302.734 35296.795)


In [37]:
# Check type
exits_df.crs

<Projected CRS: PROJCS["SVY21",GEOGCS["GCS_WGS_1984",DATUM["WGS_19 ...>
Name: SVY21
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS_84
- Prime Meridian: Greenwich

In [38]:
# SVY21 is EPSG 3414
# we convert this to 4326
exits_df = exits_df.to_crs(4326)

# we also swap around lat longs
exits_df['geometry']=exits_df.geometry.map(lambda point: shapely.ops.transform(lambda x, y: (y, x), point))
exits_df

,EXIT_CODE,EXIT_CODE_,geometry
0,None,Exit A,POINT (1.327 103.890)
1,None,Exit B,POINT (1.326 103.891)
2,None,Exit C,POINT (1.325 103.890)
3,None,Exit B,POINT (1.389 103.886)
4,None,Exit A,POINT (1.389 103.886)
...,...,...,...
474,None,Exit E,POINT (1.335 103.962)
475,None,Exit D,POINT (1.325 103.888)
476,None,Exit G,POINT (1.335 103.963)
477,None,Exit A,POINT (1.335 103.962)


In [39]:
exits_df.to_postgis('station_exits',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})

## Load and Process Roads

In [43]:
roads_df =  gpd.read_file(roads)
roads_df

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85833 1.35595 0.00000, 103.8..."
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85759 1.35611 0.00000, 103.8..."
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.86042 1.36817 0.00000, 103.8..."
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85978 1.37228 0.00000, 103.8..."
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85937 1.36913 0.00000, 103.8..."
...,...,...,...
5130,kml_5131,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.79719 1.28108 0.00000, 103.7..."
5131,kml_5132,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.83335 1.28089 0.00000, 103.8..."
5132,kml_5133,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.81821 1.28081 0.00000, 103.8..."
5133,kml_5134,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.83353 1.28081 0.00000, 103.8..."


In [44]:
# get road details

def process_geodf(df, tag, list_of_col_names):
    add_cols = {key:[] for key in list_of_col_names}
    for i in df.iterrows():
        soup = BeautifulSoup(i[1][1]).findAll(tag)
        add_cols[list_of_col_names[0]].append(soup[0].text)
        add_cols[list_of_col_names[1]].append(soup[1].text)
    return df.assign(**add_cols)


In [45]:
roads_df = process_geodf(roads_df,'td',['name','type'])

In [46]:
roads_df

,Name,Description,geometry,name,type
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85833 1.35595 0.00000, 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85759 1.35611 0.00000, 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.86042 1.36817 0.00000, 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85978 1.37228 0.00000, 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.85937 1.36913 0.00000, 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
...,...,...,...,...,...
5130,kml_5131,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.79719 1.28108 0.00000, 103.7...",40,Layers/Contour_250K
5131,kml_5132,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.83335 1.28089 0.00000, 103.8...",20,Layers/Contour_250K
5132,kml_5133,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.81821 1.28081 0.00000, 103.8...",20,Layers/Contour_250K
5133,kml_5134,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (103.83353 1.28081 0.00000, 103.8...",20,Layers/Contour_250K


In [48]:
# we also swap around lat longs - leave Z
roads_df['geometry']=roads_df.geometry.map(lambda line: shapely.ops.transform(lambda x, y, z: (y, x, z), line))
roads_df

,Name,Description,geometry,name,type
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.356 103.858 0.000, 1.356 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.356 103.858 0.000, 1.357 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.368 103.860 0.000, 1.367 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.372 103.860 0.000, 1.371 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.369 103.859 0.000, 1.369 103.8...",CENTRAL EXPRESSWAY,Layers/Expressway_Sliproad
...,...,...,...,...,...
5130,kml_5131,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.281 103.797 0.000, 1.281 103.7...",40,Layers/Contour_250K
5131,kml_5132,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.281 103.833 0.000, 1.281 103.8...",20,Layers/Contour_250K
5132,kml_5133,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.281 103.818 0.000, 1.281 103.8...",20,Layers/Contour_250K
5133,kml_5134,<center><table><tr><th colspan='2' align='cent...,"LINESTRING Z (1.281 103.834 0.000, 1.281 103.8...",20,Layers/Contour_250K


In [49]:
roads_df.to_postgis('station_exits',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='LINESTRING Z', srid= 4326)})